### IMPLEMENTACIÓN DE FILTROS GAUSSIANOS

En este trabajo práctico se busca la implementación de un filtro gaussiano para el filtrado espectral de una imagen. La función desarrollada retorna un filtro Gaussiano de tamaño HxW y su transformada de Fourier. 

In [2]:
#Import libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import scipy.stats as st
from PIL import Image
from glob import glob

%matplotlib


Using matplotlib backend: QtAgg


   QT_AUTO_SCREEN_SCALE_FACTOR to enable platform plugin controlled per-screen factors.
   QT_SCREEN_SCALE_FACTORS to set per-screen DPI.
   QT_SCALE_FACTOR to set the application global scale factor.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


### Ejercicio 1

In [3]:
def create_gauss_filter(h, w, k_size, sigma):
    #Reading image
    img = cv.imread('./imgs/tp2/img_tp2.png', cv.IMREAD_GRAYSCALE)
    h = img.shape[0]
    w = img.shape[1]
    
    #2D Kernel generation
    x = np.linspace(-sigma, sigma, k_size+1)
    kernel1d = np.diff(st.norm.cdf(x))
    kernel2d = np.outer(kernel1d, kernel1d)
    gaussian_kernel = kernel2d/kernel2d.sum()
    
    #Padding generation
    image_padded = np.zeros((h + (gaussian_kernel.shape[0]-1), w +
                             (gaussian_kernel.shape[1]-1)))
    
    kernel_img_dim = image_padded[(gaussian_kernel.shape[0]//2):-(gaussian_kernel.shape[0]//2), (gaussian_kernel.shape[1]//2):-(gaussian_kernel.shape[1]//2)]

    #Fourier Transform for the Kernel
    kernel_fft = np.fft.fft2(kernel_img_dim)

    #Fourier Transform for the image
    img_fft = np.fft.fft2(img)

    #Multiply  img_fft * kernel_fft 
    img_kernel =  img_fft * kernel_fft

    #Fft inverse and take the real part
    img_output = np.real(np.fft.ifft2(img_kernel))
    print(img_output)

    #Graficamos los resultado
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.axis('off')
    plt.imshow(img_output)

    return img_output

In [4]:
img = cv.imread('./imgs/tp2/img_tp2.png', cv.IMREAD_GRAYSCALE)
h = img.shape[0]
w = img.shape[1]

In [5]:
create_gauss_filter(h,w,5, 3)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Ejercicio 2

In [6]:
def quality_measure(img):
    h = img.shape[0]
    w = img.shape[1]
    #Fourier Transfor for the image
    image_fft = np.fft.fft2(img)
    #Shifted image fft
    shifted_image_fft = np.fft.fftshift(image_fft)
    #Abosluted value of shifted image
    ab_shifted_image = abs(shifted_image_fft)
    #Calculate maximun value of frequency component in ab_shifted_iamge
    max_shifted_image = max(ab_shifted_image)
    #Total number of pixel > threshold
    threshold = max_shifted_image/1000
    th = np.sum(np.array(max_shifted_image)>=threshold)
    image_quality_measure = th/(h+w)

    return image_quality_measure
    
    

### Ejercicio 3

Se eligió el método de Tenengrad para la evaluación de las métricas. Este método mide el foco de la imagen basado en la magnitud del gradiente según la siguiente función:

$\phi_{x,y}$ =  $\sum_{(i,j) \in \Omega}(G_{x}(i,j)^2 + G_{y}(i,j)^2)$

Donde $G_{x}$ y $G_{y}$ son los gradientes X e Y obtnenidos al convolusionar la imagen dada con el operador de Sobel. 

In [7]:
import cv2 as cv
def tenengrad(img, ksize):
    gX = cv.Sobel(img, ddepth=cv.CV_32F, dx=1, dy=0, ksize=ksize)
    gY = cv.Sobel(img, ddepth=cv.CV_32F, dx=0, dy=1, ksize=ksize)
    
    #Convertimos el valor de punto flotante a uint8 
    gX = cv.convertScaleAbs(gX)
    gY = cv.convertScaleAbs(gY)

    # Combinamos los gradientes en una sola imagen
    combined = cv.addWeighted(gX, 0.5, gY, 0.5, 0)

    # show our output images
    cv.imshow("Sobel X", gX)
    cv.imshow("Sobel Y", gY)
    cv.imshow("Sobel Combined", combined)
    cv.waitKey(0)

    return combined

In [12]:
img = cv.imread('./imgs/tp2/img_tp2.png', cv.IMREAD_GRAYSCALE)
ksize = 3
tenengrad(img, ksize)